In [2]:
!pip3 install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 4.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 3.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.7 MB/s eta 0:00:00


In [7]:
!pip3 install pywsd

In [9]:
!pip3 install nltk

In [12]:
!pip3 install flashtext

  Preparing metadata (setup.py) ... done
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9297 sha256=b4fc675d348b7fcb7ae1b169e69cf1373f8225617493cb96addc2abca2b3ce15
  Stored in directory: /Users/khushi/Library/Caches/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext


In [14]:
!pip3 install yake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 865.1 kB/s eta 0:00:00 0:00:01


In [17]:
import streamlit as st
import requests
import json
import re
import random
import spacy
import string
import scipy
import nltk
from string import punctuation
from nltk import tokenize
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk, simple_lesk, cosine_lesk
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
import yake
import io
from IPython.display import Markdown, display
from random import shuffle
from io import StringIO
import torch

In [12]:
class Distractors:
    
    def get_distractors_wordnet(syn, word):
        distractors = []
        word = word.lower()
        orig_word = word
        if len(word.split()) > 0:
            word = word.replace(" ", "_")
        hypernym = syn.hypernyms()
        if len(hypernym) == 0:
            return distractors
        for item in hypernym[0].hyponyms():
            name = item.lemmas()[0].name()
            if name == orig_word:
                continue
            name = name.replace("_", " ")
            name = " ".join(w.capitalize() for w in name.split())
            if name is not None and name not in distractors:
                distractors.append(name)
        return distractors
    
    def get_wordsense(sent, word):
        word = word.lower()
        if len(word.split())>0 :
            word = word.replace(" ", "_")
        synsets = wn.synsets(word, 'n')
        if synsets:
            wup = max_similarity(sent, word, 'wup', pos='n')
            adapted_lesk_output = adapted_lesk(sent, word, pos='n')
            lowest_index = min (synsets.index(wup), synsets.index(adapted_lesk_output))
            return synsets[lowest_index]
        else:
            return None
        
    def get_distractors_conceptnet(word):
        word = word.lower()
        orig_word = word
        if len(word.split()) > 0:
            word = word.replace(" ", "_")
        distractors = []
        url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
        obj = requests.get(url).json()
        for edge in obj['edges']:
            link = edge['end']['term']
            url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
            obj2 = requests.get(url2).json()
            for edge in obj2['edges']:
                word2 = edge['start']['label']
                if word2 not in distractors and orig_word.lower() not in word2.lower():
                    distractors.append(word2)
        return distractors
    
    def generate_option(keyword_sentence_mapping):
        key_distractor_list = {}
        for keyword in keyword_sentence_mapping:
            wordsense = Distractors.get_wordsense(keyword_sentence_mapping[keyword][0], keyword)
            if wordsense:
                distractors = Distractors.get_distractors_wordnet(wordsense, keyword)
                if len(distractors) == 0:
                    distractors = Distractors.get_distractors_conceptnet(keyword)
                if len(distractors) != 0:
                    key_distractor_list[keyword] = distractors
            else:
                distractors = Distractors.get_distractors_conceptnet(keyword)
                if len(distractors) != 0:
                    key_distractor_list[keyword] = distractors
        index = 1
        st.text("*******************************************************")
        st.text("Multiple Choice Questions")
        st.write("*******************************************************")
        for each in key_distractor_list:
            sentence = keyword_sentence_mapping[each][0]
            pattern = re.compile(each, re.IGNORECASE)
            output = pattern.sub("_____", sentence)
#             st.write("%s)"%(index), output)
            st.write(f"{index} . \t\t <span style='color:blue;'>{output}<span>", unsafe_allow_html=True)
            choices = [each.capitalize()] + key_distractor_list[each]
            top4choices = choices[:4]
            answer = top4choices[0]
#             st.write("\t", top4choices[0])
            random.shuffle(top4choices)
            optionchoices = ['a', 'b', 'c', 'd']
            for idx, choice in enumerate(top4choices):
                if choice == answer:
                    st.write(f" \t\t {optionchoices[idx]} ) <span style='color:green;'>{choice}</span> ", unsafe_allow_html=True)
                else:
                    st.write(f" \t\t {optionchoices[idx]} ) <span style='color:red;'>{choice}</span> ", unsafe_allow_html=True)
            index = index + 1 